# Setup

In [1]:
import pandas as pd
import re
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Data Preparation 
Read scraped data

In [2]:
df =  pd.read_csv('Data_Boliga_20220208.csv')

df.head()

,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5.000.000 kr.,17-01-2022,Alm. Salg,93 m²,53.763 kr/m²,3 2007 -5% Aktuel værdi
1,E,"Strandvejen 8, 3. tv",2100 København Ø,8.495.000 kr.,14-01-2022,Alm. Salg,170 m²,49.971 kr/m²,6 1929 Aktuel værdi
2,E,"Nørrebrogade 9F, 1. th",2200 København N,5.400.000 kr.,14-01-2022,Alm. Salg,105 m²,51.429 kr/m²,3 1863 -2% Aktuel værdi
3,E,"Badensgade 48, 2",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,65 m²,110.769 kr/m²,3 1899 Aktuel værdi
4,E,"Badensgade 48, 1",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,79 m²,91.139 kr/m²,3 1899 Aktuel værdi


## Initial filter

Keep only observations for apartments which was soled as "almindelig salg".

In [3]:
print(df.shape)

df = df[(df['Type'] == 'E') & (df['SellType'] == 'Alm. Salg')]

print(df.shape)

(54793, 9)
(40673, 9)


## Transform data

### Text to numeric

In [4]:
df['Price'] = df['Price'].str.replace('.', '', regex = True).str.replace('kr', '', regex = True)
df['Size'] = df['Size'].str.extract('(\d+)')
df['Price_m2']  = df['Price_m2'].str.replace('.', '', regex = True).str.extract('(\d+)')
df[['Rooms', 'YearBuilt', 'PriceChange']] =  df['Rooms_YearBuilt_PriceChange'].str.split(' ', expand = True).iloc[:,0:3]
df.loc[df['PriceChange'] == 'Aktuel', ['PriceChange']] = '0'
df['PriceChange'] = df['PriceChange'].str.replace('%', '', regex  =True)

df['PostalCode'] = df['Area'].str.extract('(\d{4})')
df['AreaName'] = df['Area'].str.extract('(\D+)')

# Check first rows
df.head()


,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange,Rooms,YearBuilt,PriceChange,PostalCode,AreaName
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5000000,17-01-2022,Alm. Salg,93,53763,3 2007 -5% Aktuel værdi,3,2007,-5,2100,København Ø
1,E,"Strandvejen 8, 3. tv",2100 København Ø,8495000,14-01-2022,Alm. Salg,170,49971,6 1929 Aktuel værdi,6,1929,0,2100,København Ø
2,E,"Nørrebrogade 9F, 1. th",2200 København N,5400000,14-01-2022,Alm. Salg,105,51429,3 1863 -2% Aktuel værdi,3,1863,-2,2200,København N
3,E,"Badensgade 48, 2",2300 København S,7200000,14-01-2022,Alm. Salg,65,110769,3 1899 Aktuel værdi,3,1899,0,2300,København S
4,E,"Badensgade 48, 1",2300 København S,7200000,14-01-2022,Alm. Salg,79,91139,3 1899 Aktuel værdi,3,1899,0,2300,København S


Convert numeric columns from characters to numeric. Note the change for a row where an incorrect value had been registered for price change.

In [5]:
# Correct value
df.loc[df['PriceChange'] == '299.499.900', 'PriceChange'] = 0 

# Creat list with columns to convert
num_cols = ['Price', 'Size', 'Price_m2', 'Rooms', 'YearBuilt', 'PriceChange', 'PostalCode']

# Loop over list
for col in num_cols:
    
    df[col] = df[col].astype(int)
    
# Check data types
print(df.dtypes)

Type                           object
StreetAddress                  object
Area                           object
Price                           int32
Date                           object
SellType                       object
Size                            int32
Price_m2                        int32
Rooms_YearBuilt_PriceChange    object
Rooms                           int32
YearBuilt                       int32
PriceChange                     int32
PostalCode                      int32
AreaName                       object
dtype: object


### Floor number

Create variable for the floor number of the apartment by extracting the number from street address column, "st" stands for ground floor.

In [6]:
df['Floor'] = df['StreetAddress'].str.extract(', (\d+|st)')
df.loc[df['Floor'] == 'st', 'Floor'] = 0

Check if any null values appeared for the floor number.

In [7]:
df_floor_null = df[df['Floor'].isnull()]

print(df_floor_null.shape)

df_floor_null.head(15)

(433, 15)


,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange,Rooms,YearBuilt,PriceChange,PostalCode,AreaName,Floor
48,E,Overbys Allé 1A,2500 Valby,7550000,05-01-2022,Alm. Salg,121,62397,6 1900 -6% Aktuel værdi,6,1900,-6,2500,Valby,NaN
398,E,Strandgade 10A,1401 København K,11200000,01-12-2021,Alm. Salg,216,51852,5 1720 Aktuel værdi,5,1720,0,1401,København K,NaN
647,E,Øresundsvej 132D,2300 København S,6995000,13-11-2021,Alm. Salg,132,52992,4 1934 Aktuel værdi,4,1934,0,2300,København S,NaN
724,E,Norgesmindevej 35,2900 Hellerup,11350000,10-11-2021,Alm. Salg,223,50897,5 1908 Aktuel værdi,5,1908,0,2900,Hellerup,NaN
777,E,Kirkebjerg Allé 38,2720 Vanløse,2555000,07-11-2021,Alm. Salg,59,43305,2 1903 -2% Aktuel værdi,2,1903,-2,2720,Vanløse,NaN
862,E,Øresundsvej 132C,2300 København S,5995000,02-11-2021,Alm. Salg,132,45417,4 1934 Aktuel værdi,4,1934,0,2300,København S,NaN
992,E,Prøvestens Allé 3A,2300 København S,2870000,27-10-2021,Alm. Salg,74,38784,2 1900 -4% Aktuel værdi,2,1900,-4,2300,København S,NaN
1586,E,Rosenvængets Allé 5A,2100 København Ø,3750000,28-09-2021,Alm. Salg,59,63559,2 1875 Aktuel værdi,2,1875,0,2100,København Ø,NaN
1812,E,Øresundsvej 126A,2300 København S,2895000,21-09-2021,Alm. Salg,54,53611,2 1920 -3% Aktuel værdi,2,1920,-3,2300,København S,NaN
1865,E,Constantin Hansens Gade 8C,1799 København V,3120000,19-09-2021,Alm. Salg,42,74286,1 2019 -2% Aktuel værdi,1,2019,-2,1799,København V,NaN


There are 433 observations where a floor number can not be extracted. A quick check on some of the addresses shows that the type column is either coded incorrectly or the observations is an apartment located in a villa. Keep the obersvations for now, setting the floor variable to -1 for easy identification later.

In [8]:
df.loc[df['Floor'].isnull(), 'Floor'] = -1

df['Floor'] = df['Floor'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40673 entries, 0 to 54792
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Type                         40673 non-null  object
 1   StreetAddress                40673 non-null  object
 2   Area                         40673 non-null  object
 3   Price                        40673 non-null  int32 
 4   Date                         40673 non-null  object
 5   SellType                     40673 non-null  object
 6   Size                         40673 non-null  int32 
 7   Price_m2                     40673 non-null  int32 
 8   Rooms_YearBuilt_PriceChange  40673 non-null  object
 9   Rooms                        40673 non-null  int32 
 10  YearBuilt                    40673 non-null  int32 
 11  PriceChange                  40673 non-null  int32 
 12  PostalCode                   40673 non-null  int32 
 13  AreaName                     40

## Select relevant columns

Drop redundant columns

In [9]:
df.drop(['Rooms_YearBuilt_PriceChange', 'Type', 'SellType', 'Area'], inplace = True, axis = 1)

Insert longitude and latitude in data frame and inspect the results

# Filter data
The data need to be filtered for outliers and possible incorrections. By checking summary statistics it is possible a couple if things.
- There are a large spread in values for price, size and price per sqm.
<br>
<br>
- The minimum value for YearBuilt is year 0, which seems odd.
<br>
<br>
- A price change o -100 or +249% does not seem realistic.


In [ ]:
df.describe()

In [ ]:
df.hist(bins=100, figsize=(12,10))

Check some boxplots to identify possible outliers

In [ ]:
fig = plt.figure(figsize = (13,8))


plt.subplot(2,2,1)
sns.boxplot(data = df['Price'])
plt.subplot(2,2,2)
sns.boxplot(data = df['Size'])
plt.subplot(2,2,3)
sns.boxplot(data= df['Price_m2'])
plt.subplot(2,2,4)
sns.boxplot(data = df['PriceChange'])

### Price vs size

There seems to be quite some spread in the variables, for example a maximum price of 85,000,000 DKK or a maximum size of 857 square meters. Let's to plot the relationship between price and size.

In [ ]:
sns.scatterplot(data = df,
            x = 'Size',
            y = 'Price',
            alpha = 0.1)

The horizontal lines looks odd, especially considering it appears for the most expensive. Let's look more into those observations.

In [ ]:
df.loc[df['Price'] > 80000000].head(10)

It seems like these are apartments bought in larger housing investments, which is probably the case for the other data points on the horizontal lines as well. By counting number of observations by street address (excluding floor and apartment), price, and date, I can find similar instances, which most probably are not private acqusitions.

In [ ]:
df['Street'] =  df['StreetAddress'].str.extract("([^0-9]+\d{0,3})")

df_mult_out = (df.groupby(['Price', 'Date', 'Street'])['Price'].size()
               .sort_values(ascending = False)
               .reset_index(name = 'Count')
               .query('Count >= 2')
              )

df_mult_out.head(10)

Lets filter out the observations being part of these investments, check how many observations that are filtered out. 

In [ ]:
print(df_mult_out['Count'].sum())

# Join number of counts to original data frame nad filter out rows 
df = (
    df.merge(
    df_mult_out,
    how = 'left',
    on = ['Price', 'Date', 'Street'])
    .query('Count.isna()', engine = 'python')
)

print(df.shape)
df.head()

df.drop('Count', axis = 1, inplace = True)

Plot the price vs. size to see how the filtering has changed the data.

In [ ]:
sns.scatterplot(data = df,
            x = 'Size',
            y = 'Price',
            alpha = 0.1)

In [ ]:
fig = plt.figure(figsize = (13,8))


plt.subplot(2,2,1)
sns.boxplot(data = df['Price'])
plt.subplot(2,2,2)
sns.boxplot(data = df['Size'])
plt.subplot(2,2,3)
sns.boxplot(data= df['Price_m2'])
plt.subplot(2,2,4)
sns.boxplot(data = df['PriceChange'])


### Size

There still seem to be some outliers with regards to size. Start by visualizing the distribution of apartment sizes by a box plot and again the relationship between price and size.

In [ ]:
sns.boxplot(data = df['Size'])


df[['Size', 'Price_m2']].describe()

In [ ]:
sns.scatterplot(data = df,
                x = 'Size',
                y = 'Price',
                alpha = 0.3
               )

In [ ]:
df['Price_m2_norm'] = (df['Price_m2'] - df['Price_m2'].mean()) / df['Price_m2'].std()

print(df['Price_m2_norm'].describe())

sns.scatterplot(data = df,
                x = 'Size',
                y = 'Price',
                hue='Price_m2_norm', 
                palette='RdBu',
                #alpha = 0.3
               )




In [ ]:
px.scatter(df,
           x = 'Size',
           y = 'Price',
           color = 'Price_m2_norm'
          )

Check the most extreme cases

In [ ]:
df.sort_values(by = 'Price_m2_norm')

The two observations around 850 and 400 m2 looks like outliers 

In [ ]:
max_size = 200
df_ls = df[df['Size'] > max_size]


In [ ]:
df_ls.hist(bins = 50,
           figsize=(12,10))





Write data to new file

In [ ]:
df.to_csv('Data_Boliga_20220208_cleaned_20220313.csv', index  = False)